In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error


In [ ]:
device = torch.device('cuda:1')
batch_size = 2048
inputdir = '../data/'

if torch.cuda.is_available():
    print("Available CUDA devices:", torch.cuda.device_count())
    for i in range(torch.cuda.device_count()):
        print(f"Device {i}: {torch.cuda.get_device_name(i)}")
else:
    print("CUDA is not available.")

In [1]:
# Load datasets from specified input directory
train_data = pd.read_csv(inputdir + 'loop_sensor_train.csv')
test_data = pd.read_csv(inputdir + 'loop_sensor_test_x.csv')
geo_data = pd.read_csv(inputdir + 'geo_reference.csv')

# Merge geographic data with train and test datasets
train_data = train_data.merge(geo_data, on='iu_ac', how='left')
test_data = test_data.merge(geo_data, on='iu_ac', how='left')

# Define adjacency matrix creation function and normalize features
def create_adjacency_matrix(data, threshold=0.01):
    coords = data[['geo_point_2d']].str.split(',', expand=True).astype(float).values
    dist_matrix = np.sqrt((coords[:, np.newaxis, :] - coords[np.newaxis, :, :]) ** 2).sum(axis=2)
    adj_matrix = (dist_matrix < threshold).astype(int)
    return torch.tensor(adj_matrix, dtype=torch.float32).to(device)

# Create adjacency matrix and move it to the device
adjacency_matrix = create_adjacency_matrix(train_data)

# Normalize features
scaler = StandardScaler()
train_features = scaler.fit_transform(train_data[['t_1h', 'etat_barre']])
train_labels = train_data['q'].values
train_features = torch.tensor(train_features, dtype=torch.float32).to(device)
train_labels = torch.tensor(train_labels, dtype=torch.float32).unsqueeze(1).to(device)


NameError: name 'pd' is not defined

In [ ]:
class GraphConvolution(nn.Module):
    def __init__(self, in_features, out_features):
        super(GraphConvolution, self).__init__()
        self.fc = nn.Linear(in_features, out_features, bias=False)

    def forward(self, x, adj):
        x = torch.matmul(adj, x)
        x = self.fc(x)
        return torch.relu(x)

class TemporalAttentionLayer(nn.Module):
    def __init__(self, num_steps):
        super(TemporalAttentionLayer, self).__init__()
        self.W = nn.Parameter(torch.randn(num_steps))

    def forward(self, x):
        beta = torch.softmax(self.W, dim=0)
        return torch.matmul(x, beta)

class ASTGCN(nn.Module):
    def __init__(self, num_features, num_nodes):
        super(ASTGCN, self).__init__()
        self.gconv1 = GraphConvolution(num_features, 16)
        self.gconv2 = GraphConvolution(16, 32)
        self.temporal_attention = TemporalAttentionLayer(num_steps=24)  # Assuming hourly data for one day
        self.fc = nn.Linear(32, 1)

    def forward(self, x, adj):
        x = self.gconv1(x, adj)
        x = self.gconv2(x, adj)
        x = self.temporal_attention(x)
        x = self.fc(x)
        return x

# Instantiate and move model to the specified device
model = ASTGCN(num_features=2, num_nodes=len(train_data['iu_ac'].unique())).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = nn.MAELoss()


In [ ]:
# Define a simple Dataset class for your data
class TrafficDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Create dataset and data loader
train_dataset = TrafficDataset(train_features, train_labels)
train_loader = DataLoader(train_dataset, batch_size=2048, shuffle=True)

# Training loop
for epoch in range(50):
    model.train()
    total_loss = 0
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(features, adjacency_matrix)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss / len(train_loader)}')


In [ ]:
model.eval()
test_features = scaler.transform(test_data[['t_1h', 'etat_barre']])
test_features = torch.tensor(test_features, dtype=torch.float32).to(device)
test_preds = model(test_features, adjacency_matrix)
test_preds = test_preds.detach().cpu().numpy()

# Prepare submission
submission = pd.DataFrame({
    'iu_ac': test_data['iu_ac'],
    't_1h': test_data['t_1h'],
    'etat_barre': test_data['etat_barre'],
    'estimate_q': test_preds.flatten()
})
submission.to_csv(inputdir + 'submission.csv', index=False)
